<a href="https://colab.research.google.com/github/anudaweerasinghe/evo-genome-classification/blob/add-naive-training/distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install transformers datasets evaluate accelerate

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install numpy --upgrade

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [17]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
genomes = load_dataset("anudaw/genome-classification")
# genomes = load_dataset("imdb")

In [5]:
id2label = {0: "H", 1: "I", 2: "C"}
label2id = {"H" : 0, "I": 1, "C": 2}


In [6]:
def update_label_to_int(examples):
    return {"seq" : examples["seq"], 'label': [label2id[label] for label in examples['label']]}

genomes = genomes.map(update_label_to_int, batched=True)
print(genomes['train'][0])

{'seq': 'ATCTAGAATATAGAAAAGTGGTAAGACAAAAAAGAATAAACTGGTTATTTGATAGAATAAGAGAAAGAGCAGAAGACAGTGGCAATGAGAGCGAGGGAGATACAGAGGAATTGGCAGCACTTGGGGAAACGGGGCATCTTATTCTTGGGGATATTAATAATCTGTAGCGCTGCAAACAACTTGTGGGTCACAGTCTATTATGGGGTACCTGTGTGGAAAGAAGCAACAACCACTCTATTCTGTGCATCAGATGCTAAAGCATATGAAACAGAGGTACATAATGTCTGGGCTACACATGCCTGTGTACCCACGGACCCCAACCCACAAGAAGTAGTTTTGGAAAATGTAACAGAAAATTTTAACATGTGGAAAAATAACATGGTAGAGCAGATGCATACAGATATAATCAGTTTATGGGATGAAAGCCTAAAACCATGTGTAAAGTTAACCCCTCTCTGTGTCACTCTAACTTGTACCAATGTAACTAATAACCGTACCAATGCAAATAAGAACGATACCAATATCAATGCAACTGTGACATCAACTGACGAGATAAAAAATTGCTCTTTCAATATAACCACAGAACTAAAAGATAAGAAAAAGAGAGTGTCTGCACTTTTCTATAAGCTTGATATAGTGCAAATAAAGCAATCTGAAATTAATCAATCTGAATCTGAAGATAGGCTAATAAATTGTAATACCTCAACCGTTACACAGGCCTGTCCAAAGGTATCCTTTGAGCCAATTCCCATACATTATTGTGCTCCAGCTGGCTTTGCGATTCTAAAGTGTAATAATAACACATGCAATGGAACAGGGCCATGCACAAATGTCAGCACAGTACAATGTACACATGGAATTAAACCAGTGGTATCAACTCAATTGCTATTAAATGGCAGCCTAGCAGAAGAAGAGATAATTATTAGGTCTGAAGATATTACAAAGAATACAAAAAACATAATAGTACAGCTTAATGAAGCTGTAGAAATTA

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/canine-c")

In [8]:
def preprocess_function(examples):
    # print(len(examples['seq']))
    return tokenizer(examples["seq"], truncation=True)

tokenized_genomes = genomes.map(preprocess_function, batched=True)

In [9]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
import evaluate
accuracy = evaluate.load("accuracy")

In [15]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return accuracy.compute(predictions=predictions, references=labels)



In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "google/canine-c", num_labels=3, id2label=id2label, label2id=label2id
)



/home/addykan/.local/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-c and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir = "model_data",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_genomes["test"],
    eval_dataset=tokenized_genomes["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()
trainer.push_to_hub()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.010600,0.155503,0.976167
2,0.000100,0.000244,0.999979


CommitInfo(commit_url='https://huggingface.co/addykan/model_data/commit/8f67ded2f3442a64f50f2002877c514f18dfdc6a', commit_message='End of training', commit_description='', oid='8f67ded2f3442a64f50f2002877c514f18dfdc6a', pr_url=None, pr_revision=None, pr_num=None)